In [18]:
import pandas as pd
import os

data_dir = os.path.join('..', 'data')

labels_df = pd.read_csv(os.path.join(data_dir, 'labels.csv'), 
                       header=None, 
                       names=['frame', 'exercise'])

landmarks_df = pd.read_csv(os.path.join(data_dir, 'landmarks.csv'))

print("Labels shape:", labels_df.shape)
print("Landmarks shape:", landmarks_df.shape)

Labels shape: (8408, 2)
Landmarks shape: (277431, 5)


# What exercises was performed? How was it perfomed?

The training session consisted of upper body exercises performed in a university class setting. All exercises were conducted in a stationary standing position using resistance bands.
Exercise Structure:
* Each exercise was performed for 5 repetitions
* Total of 16 different exercise variations
* Some exercises were repeated with 3 different angles/variations to target muscles differently
* Exercises were performed consecutively (back-to-back)

In [19]:
labels_df.head(2)

,frame,exercise
0,0,-1
1,1,-1


In [20]:
landmarks_df.head(2)

,frame_number,landmark,x,y,z
0,0,NOSE,0.415256,0.361049,-0.164153
1,0,LEFT_EYE_INNER,0.422111,0.350810,-0.140611


# How it was collected and whats the data inside mean?

The participant performed exercises while wearing sensors positioned at the elbows, wrists, and hip area. Additionally, video recordings were enhanced using MediaPipe to track key body points.

The labels_df contains video frames and corresponding exercise labels
* Label values range from -1 to 15:
* * -1 indicates no exercise being performed at that moment
* * 0-15 represent individual exercises
* Labels were manually annotated using a separate program developed during the university classes

The landmarks_df contains:
* Frame numbers from the video recording
* Position of each MediaPipe keypoint in X, Y, Z coordinates (3D space)


### Dropping face, legs and finger keypoints as they are not necessary for movement classification

In [21]:
head_landmarks = [
    'NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER',
    'RIGHT_EYE_INNER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER',
    'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT',
    # Adding leg landmarks
    'LEFT_KNEE', 'RIGHT_KNEE',
    'LEFT_ANKLE', 'RIGHT_ANKLE',
    'LEFT_HEEL', 'RIGHT_HEEL',
    'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX',
    # Adding finger landmarks
    'LEFT_PINKY', 'RIGHT_PINKY',
    'LEFT_INDEX', 'RIGHT_INDEX', 
    'LEFT_THUMB', 'RIGHT_THUMB'
]

landmarks_df = landmarks_df[~landmarks_df['landmark'].isin(head_landmarks)]

In [22]:
def process_landmarks_data(df):
    pivoted_df = df.pivot(index='frame_number', 
                         columns='landmark', 
                         values=['x', 'y', 'z'])
    
    pivoted_df.columns = [f'{landmark}_{coord}' 
                         for coord, landmark in pivoted_df.columns]
    
    return pivoted_df.reset_index()

In [23]:
processed_landmarks_df = process_landmarks_data(landmarks_df)

In [24]:
df = pd.merge(processed_landmarks_df, 
                    labels_df,
                    left_on='frame_number',
                    right_on='frame',
                    how='left')

if 'frame' in df.columns:
    df = df.drop('frame', axis=1)

In [25]:
df['exercise'] = pd.to_numeric(df['exercise'])

In [26]:
df.head(5)

,frame_number,LEFT_ELBOW_x,LEFT_HIP_x,LEFT_SHOULDER_x,LEFT_WRIST_x,RIGHT_ELBOW_x,RIGHT_HIP_x,RIGHT_SHOULDER_x,RIGHT_WRIST_x,LEFT_ELBOW_y,...,RIGHT_WRIST_y,LEFT_ELBOW_z,LEFT_HIP_z,LEFT_SHOULDER_z,LEFT_WRIST_z,RIGHT_ELBOW_z,RIGHT_HIP_z,RIGHT_SHOULDER_z,RIGHT_WRIST_z,exercise
0,0,0.494306,0.435416,0.461974,0.492193,0.320912,0.381529,0.367181,0.322582,0.507141,...,0.502203,-0.089719,0.000337,0.039939,-0.346921,-0.075649,-0.000382,0.042103,-0.336874,-1
1,1,0.492015,0.435435,0.462259,0.491906,0.320943,0.381469,0.366824,0.320433,0.521023,...,0.503574,-0.093571,0.000344,0.051836,-0.364749,-0.065798,-0.000387,0.052328,-0.318225,-1
2,2,0.492051,0.435449,0.462260,0.491421,0.320974,0.381461,0.366673,0.319015,0.517668,...,0.498068,-0.108150,0.000348,0.058875,-0.375957,-0.076405,-0.000389,0.058376,-0.319684,-1
3,3,0.491493,0.435497,0.462321,0.492260,0.320294,0.381467,0.366689,0.318692,0.525453,...,0.488508,-0.073606,0.001731,0.072559,-0.326744,-0.084063,-0.001726,0.055875,-0.333625,-1
4,4,0.491555,0.435549,0.462301,0.493636,0.318972,0.381528,0.366703,0.318285,0.523011,...,0.481967,-0.078110,0.002196,0.068983,-0.335180,-0.097579,-0.002175,0.054835,-0.344045,-1


In [27]:
df.to_csv(os.path.join('..', 'data', 'data.csv'), index=False)

### Data informations

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8407 entries, 0 to 8406
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   frame_number      8407 non-null   int64  
 1   LEFT_ELBOW_x      8407 non-null   float64
 2   LEFT_HIP_x        8407 non-null   float64
 3   LEFT_SHOULDER_x   8407 non-null   float64
 4   LEFT_WRIST_x      8407 non-null   float64
 5   RIGHT_ELBOW_x     8407 non-null   float64
 6   RIGHT_HIP_x       8407 non-null   float64
 7   RIGHT_SHOULDER_x  8407 non-null   float64
 8   RIGHT_WRIST_x     8407 non-null   float64
 9   LEFT_ELBOW_y      8407 non-null   float64
 10  LEFT_HIP_y        8407 non-null   float64
 11  LEFT_SHOULDER_y   8407 non-null   float64
 12  LEFT_WRIST_y      8407 non-null   float64
 13  RIGHT_ELBOW_y     8407 non-null   float64
 14  RIGHT_HIP_y       8407 non-null   float64
 15  RIGHT_SHOULDER_y  8407 non-null   float64
 16  RIGHT_WRIST_y     8407 non-null   float64


In [29]:
df.head(2)

,frame_number,LEFT_ELBOW_x,LEFT_HIP_x,LEFT_SHOULDER_x,LEFT_WRIST_x,RIGHT_ELBOW_x,RIGHT_HIP_x,RIGHT_SHOULDER_x,RIGHT_WRIST_x,LEFT_ELBOW_y,...,RIGHT_WRIST_y,LEFT_ELBOW_z,LEFT_HIP_z,LEFT_SHOULDER_z,LEFT_WRIST_z,RIGHT_ELBOW_z,RIGHT_HIP_z,RIGHT_SHOULDER_z,RIGHT_WRIST_z,exercise
0,0,0.494306,0.435416,0.461974,0.492193,0.320912,0.381529,0.367181,0.322582,0.507141,...,0.502203,-0.089719,0.000337,0.039939,-0.346921,-0.075649,-0.000382,0.042103,-0.336874,-1
1,1,0.492015,0.435435,0.462259,0.491906,0.320943,0.381469,0.366824,0.320433,0.521023,...,0.503574,-0.093571,0.000344,0.051836,-0.364749,-0.065798,-0.000387,0.052328,-0.318225,-1


### Drop column and save data for model

In [30]:
df = df.drop(columns='frame_number')
df.to_csv(os.path.join('..', 'data', 'data_for_model.csv'), index=False)